# F1 Fantasy Program

## ReadMe

The goal of this program is to webscrape, and gather, F1 data. The data would then be saved, transformed, and analysed.

## Setup

In [1]:
from datetime import datetime
from bs4 import BeautifulSoup
import pandas as pd
import requests
import glob

import fastf1

### Enable Caching

In [2]:
fastf1.Cache.enable_cache('cache')

### Ignore Warnings

In [3]:
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)

# Data Aquisition

## Webscraping

In [4]:
# # url = 'https://f1fantasytools.com/driver-constructor-info'
# url = 'https://www.f1fantasytracker.com/prices.html'
# response = requests.get(url)
# soup = BeautifulSoup(response.content, 'html.parser')
# # tables = soup.find_all('table')

# current_prices = []
# driver_table = soup.find('table', id='driverTable')
# for row in table.tbody.find_all('tr'):
#     cols = row.find_all('td')
#     print(cols)
#     break

# constructor_table = soup.find('table', id='constructorTable')


In [5]:
from bs4 import BeautifulSoup
import requests

url = 'https://www.f1fantasytracker.com/prices.html'  # replace with the URL of the website you want to scrape
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

current_price = soup.find('span', {'id': 'CurrentPrice1'}).text
season_price = soup.find('span', {'id': 'SeasonPrice1'}).text

print(current_price)
print(season_price)

## API

### Get the completed events of the season (year)

In [6]:
def get_completed_events(season, b_print=False):
    schedule = fastf1.get_event_schedule(season)
    # print(schedule)
    completed_events = []
    for event in schedule['RoundNumber'][1:]:
        if schedule['EventDate'][event] < datetime.now():
            completed_events.append(schedule.iloc[[event]])        
    return completed_events

### Load the data of a completed session

In [7]:
def get_session_data(season, event_num, session_type):
    df_session = fastf1.get_session(season, event_num, session_type)
    df_session.load()
    return df_session

### Load the data of all the completed sessions

In [8]:
# def get_completed_sessions(season, completed_events, session_type):
#     completed_sessions = []
#     for i, event in enumerate(completed_events):
#         type(completed_events)
#         session = get_session_data(season, event['RoundNumber'], session_type)
#         completed_sessions.append(session)
#     return completed_sessions

In [9]:
def get_completed_sessions(season, completed_events, session_type):
    # Returns a list of dataframes
    completed_sessions = []

    for event_df in completed_events:
        for _, event in event_df.iterrows():
            session = get_session_data(season, event['RoundNumber'], session_type)
            completed_sessions.append(session)

    return completed_sessions


## Save sessions to csv

In [10]:
def save_df_list_to_csv(dataframe_list, name="dataframe"):
    for i, dataframe in enumerate(dataframe_list):
        file_name = name + '_' + str(i) + '.csv'
        dataframe.to_csv(file_name)

## TEST

In [11]:
season = 2023
event_num = 3
session_type = 'R'

session_data = get_session_data(season, event_num, session_type)
print(session_data)

completed_events = get_completed_events(season)
# print(completed_events)
completed_sessions = get_completed_sessions(season, completed_events, session_type) 
print(completed_sessions[1])

core           INFO 	Loading data for Australian Grand Prix - Race [v2.3.1]
api            INFO 	Using cached data for driver_info


KeyboardInterrupt: 

In [ ]:
results_list = [session.results for session in completed_sessions]
save_df_list_to_csv(results_list)
print(results_list)

[   DriverNumber BroadcastName Abbreviation         TeamName TeamColor  \
1             1  M VERSTAPPEN          VER  Red Bull Racing    3671C6   
11           11       S PEREZ          PER  Red Bull Racing    3671C6   
14           14      F ALONSO          ALO     Aston Martin    358C75   
55           55       C SAINZ          SAI          Ferrari    F91536   
44           44    L HAMILTON          HAM         Mercedes    6CD3BF   
18           18      L STROLL          STR     Aston Martin    358C75   
63           63     G RUSSELL          RUS         Mercedes    6CD3BF   
77           77      V BOTTAS          BOT       Alfa Romeo    C92D4B   
10           10       P GASLY          GAS           Alpine    2293D1   
23           23       A ALBON          ALB         Williams    37BEDD   
22           22     Y TSUNODA          TSU       AlphaTauri    5E8FAA   
2             2    L SARGEANT          SAR         Williams    37BEDD   
20           20   K MAGNUSSEN          MAG     Haa

## Connect to MySQL f1stats Schema

In [14]:
import mysql.connector
import json

with open('private_config.json', 'r') as file:
	config_data = json.load(file)


mydb = mysql.connector.connect(
  host=config_data['db_host'],
  user=config_data['db_user'],
  password=config_data['db_password'],
  database=config_data['db_database']
)

mycursor = mydb.cursor()

In [ ]:
insertStmt = '''
INSERT INTO teams
(name, points)
VALUES
('Asiton Martin', 10000)
;
'''
mycursor.execute(insertStmt)

mydb.commit()

selectStmt = '''
SELECT * FROM teams
;
'''
mycursor.execute(selectStmt)

results = mycursor.fetchall()
for row in results:
	print(row)

(1, 'AlphaTauri', 3)
(2, 'Haas', 1)
(3, 'McLaren', 5)
(4, 'Williams', 7)
(5, 'Red Bull Racing', 800)
(6, 'Aston Martin', 1000)
(7, 'Asiton Martin', 10000)
